Gets the training data for the ML model.
Be sure to specify the correct paths to the directories, the API key for bing maps, and the regions for Bing Maps/Overpass Turbo.

NOTE: Ensure 'requests' is installed for Python3 by running ```pip install requests``` in your machine's terminal.

In [ ]:
BingMapsAPIKey = ""     # Bing Maps API key         
TrainingImagesPath = "bing_maps_images_min_max" # Path to training images
TrainingMaskPath = ""   # Path to training masks

In [4]:
'''
Query Overpass Turbo for features in a defined bounding box
Feature: Surface Level Parking lots
Bounding Box: Indianapolis, IN

Data stored in data.json
'''
import requests
import json
import os

# Overpass API URL
overpass_url = "http://overpass-api.de/api/interpreter"

# Your Overpass query
overpass_query = """
[out:json][timeout:100];
// gather results
(
  // query part for: “parking=surface”
  node["parking"="surface"](39.7042,-86.3994,39.9569,-86.0172);
  way["parking"="surface"](39.7042,-86.3994,39.9569,-86.0172);
  relation["parking"="surface"](39.7042,-86.3994,39.9569,-86.0172);
);
// print results
out body;
>;
out skel qt;
"""

# Send the request
overpass_response = requests.get(overpass_url, params={'data': overpass_query})

# Check for HTTP errors
overpass_response.raise_for_status()

# Parse the JSON response
data = overpass_response.json()

# Save the data to a file
with open('data.json', 'w') as file:
    json.dump(data, file, indent=4)

In [3]:
'''
Clean the data and store in coordinates.json
Necessary for getting images of the feature from Bing Maps
'''
def process_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Create a dictionary to map node IDs to their lat/lon
    node_dict = {}
    for element in data['elements']:
        if element['type'] == 'node':
            node_dict[element['id']] = (element['lat'], element['lon'])

    output_data = {}
    for element in data['elements']:
        if element['type'] == 'way':
            way_id = str(element['id'])
            node_ids = element['nodes']

            # Initialize min and max values
            min_lat = min_lon = float('inf')
            max_lat = max_lon = float('-inf')
            total_lat = total_lon = 0
            coordinate_list = []

            for node_id in node_ids:
                if node_id in node_dict:
                    lat, lon = node_dict[node_id]
                    min_lat = min(min_lat, lat)
                    max_lat = max(max_lat, lat)
                    min_lon = min(min_lon, lon)
                    max_lon = max(max_lon, lon)
                    total_lat += lat
                    total_lon += lon
                    coordinate_list.append(f"{lat},{lon}")

            center_lat = total_lat / len(node_ids)
            center_lon = total_lon / len(node_ids)
            coordinates = "_".join(coordinate_list)

            output_data[way_id] = {
                "min_lat": min_lat,
                "min_lon": min_lon,
                "max_lat": max_lat,
                "max_lon": max_lon,
                "center_point_lat": center_lat,
                "center_point_lon": center_lon,
                "coordinates": coordinates
            }

    return output_data

# Store cleaned data in data.json
processed_data = process_json('data.json')

# To save the processed data to a new file
with open('coordinates.json', 'w') as file:
    json.dump(processed_data, file, indent=4)

In [ ]:
'''
Fetch images of the feature from Bing Maps

NOTE: Define your api key
'''
# Define the base API URL
base_url = "https://dev.virtualearth.net/REST/v1/Imagery/Map/Aerial/"

# Define the query parameters that are common to all requests
common_params = {
    "mapSize": "500,500",
    "mapLayer": "Basemap,Buildings",
    "key": f"{BingMapsAPIKey}",
}

# Load the dataset of IDs and their corresponding coordinates from min_max_coords.json
with open("coordinates.json", "r") as json_file:
    coordinates_data = json.load(json_file)

# Create a directory to store the images
os.makedirs(TrainingImagesPath, exist_ok=True)

# Iterate through the dataset and make API requests
for id, coords in coordinates_data.items():
    # Define unique parameters for each request
    params = {
        **common_params,
        "mapArea": f"{coords['min_lat']},{coords['min_lon']},{coords['max_lat']},{coords['max_lon']}",
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    # Check for a successful response
    if response.status_code == 200:
        # Save the image data to a file
        image_file = os.path.join(TrainingImagesPath, f"image_{id}.jpg")
        with open(image_file, "wb") as file:
            file.write(response.content)
        print(f"Image for ID {id} saved as {image_file}")
    else:
        # Print an error message if the request was not successful
        print(f"HTTP request for ID {id} failed with status code: {response.status_code}")